In [1]:
# Import libraries 
import pandas as pd
import numpy as np
#import warnings
import seaborn as sns
import matplotlib.pyplot as plt
#from joblib import Parallel, delayed
#import multiprocessing
#from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import MinMaxScaler #, StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
import math
#from sklearn.model_selection import KFold
#from sklearn.model_selection import TimeSeriesSplit
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from keras.callbacks import EarlyStopping

# Data Collection (Coleta de Dados) #

In [ ]:
# Import databases

sales_df = pd.read_excel('C:\\Users\\sabrina\\Desktop\\Faturamento.xlsx') # Collecting data from an Excel file that contains sales history information since May-2019
pd.set_option('display.max_columns', None)
sales_df.head(5)

In [ ]:
sales_df.shape

In [ ]:
# Deleting rows with information not relevant to the analysis
# The exclusion of these rows ensures that we have access only to information related to sales, disregarding possible returns, order refunds, and others.
sales_df = sales_df.drop(sales_df[sales_df['TpDocVend.'] != 'Z000'].index)
sales_df = sales_df.drop(sales_df[sales_df['G~Ctg.NF'] != '1N'].index)
sales_df = sales_df.drop(sales_df[sales_df['G~Estornado'] == 'X'].index)
sales_df = sales_df[sales_df['Material'] != '99000100'] # scrap

sales_df.shape

# Exploratory Data Analysis (EDA) (Análise Exploratória dos Dados) #

In [ ]:
# General information about the dataframe
sales_df.info()

In [ ]:
# Check if there is any missing data
sns.heatmap(sales_df.isnull(), yticklabels= False, cbar= False, cmap= 'Blues')

The heatmap shows us the correct information about the column 'G~Estornado'. This column should have all values empty, as it indicates information about invoice reversals.  
Empty values in the column 'B~Tp.aval.' do not impact our analysis, as this column will be disregarded. This column refers to material characteristics and not to revenue.

In [ ]:
# Creation of a new column with the year from the column with the sales date information 'F~Dt.fatur.'
sales_df['Year'] = pd.to_datetime(sales_df['F~Dt.fatur.']).dt.year

# Group by year and sum the quantity of items already invoiced.
yearly_sales = sales_df.groupby('Year')['Qtd.faturd'].sum()

# Creating a bar chart for the aggregated data. This chart will show the evolution of the quantities of items sold over the years.
plt.figure(figsize=(10, 6))
plt.bar(yearly_sales.index, yearly_sales.values)
plt.title('The Annual Sum of the Quantity of Invoiced Items')
plt.xlabel('Year')
plt.ylabel('Invoiced Quantity')
plt.xticks(rotation=45)  # Rotation of the x-axis labels for better visualization
plt.show()


In [ ]:
# Creation of a new column with the month from the column with the sales date information 'F~Dt.fatur.'
sales_df['month'] = pd.to_datetime(sales_df['F~Dt.fatur.']).dt.month

# Creation of the heatmap to visualize monthly sales data over the years
heatmap_data = sales_df.pivot_table(values='Qtd.faturd', index='month', columns='Year', aggfunc='sum')
plt.figure(figsize=(12, 8))
sns.heatmap(heatmap_data, annot=True, fmt=".0f", cmap='coolwarm')
plt.title('Monthly Sales Heatmap by Year')
plt.xlabel('Year')
plt.ylabel('Month')
plt.yticks(ticks=range(0, 12), labels=['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

# Data Preparation (Preparação dos Dados) #

In [9]:
# Function to normalize the material data
def normalize_material(df, column_name):
    df[column_name] = df[column_name].astype(str).str.strip().str.upper()  # Converts to string, removes spaces, and converts to uppercase
    df[column_name] = df[column_name].str.zfill(11)  # Pads with zeros on the left to ensure 11 digits
    return df

# Normalizing the specific column of each DataFrame
sales_df = normalize_material(sales_df, 'Material')

In [10]:
# Creation of a new column with the year and month from the column with the sales date information 'F~Dt.fatur.'
sales_df['YearMonth'] = pd.to_datetime(sales_df['F~Dt.fatur.']).dt.to_period('M')

In [ ]:
sales_df

In [12]:
# Keeping only the columns necessary for the analysis
'''
Material - Column containing the part number information
Qtd.faturd - Column containing the invoiced quantity per part number
YearMonth - Column containing the invoicing month and year information
'''
sales_df = sales_df[['Material', 'Qtd.faturd', 'YearMonth']]

In [ ]:
# Pivot sales data by material and date, filled missing values
sales_df = sales_df.pivot_table(index='Material', columns='YearMonth', values='Qtd.faturd', aggfunc='sum')
sales_df = sales_df.fillna(0)
sales_df

In [ ]:
# Function to remove outliers from a row (data series) and replace with mean
def remove_outliers_and_replace_with_mean(row):
    Q1 = row.quantile(0.25)
    Q3 = row.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Apply filter to find non-outlier values
    valid_values = (row >= lower_bound) & (row <= upper_bound)
    
    # Calculate the mean of the non-outlier values
    mean_valid_values = row[valid_values].mean()
    
    # Replace outliers with the calculated mean of valid values
    row[~valid_values] = mean_valid_values
    
    return row

# Applying the function remove_outliers_and_replace_with_mean on each row of the DataFrame
sales_df = sales_df.apply(remove_outliers_and_replace_with_mean, axis=1)


In [ ]:
sales_df

The DataFrame sales_df now has outliers replaced by the mean of the non-outlier values for each row.

In [ ]:
# Number of months to be used for testing
num_months_for_test = 12

# Total number of month columns in the dataframe
total_columns = sales_df.shape[1]

# Calculation of columns for training
num_columns_for_train = total_columns - num_months_for_test

# Split dataset into training and test sets
all_months = sales_df.columns[0:]
training_set = sales_df[all_months[:-num_months_for_test]].fillna(0).values # Selects all columns representing months, except the last 12 (number defined in num_months_for_test)
test_set = sales_df[all_months[-num_months_for_test:]].fillna(0).values # Selects only the last 12 month columns from the DataFrame, which correspond to the last 12 months of data.


In [ ]:
# Scaling features to range 0-1 for training and test sets
sc = MinMaxScaler(feature_range=(0,1))

training_set_scaled = sc.fit_transform(training_set)
test_set_scaled = sc.fit_transform(test_set)

# Choose Model (Escolha do Modelo) #    

Here we chose the RNN - LSTM model and defined its architecture. LSTM is a good choice because it can capture long-term patterns in time series.

In [ ]:
# Creating empty lists
X_train = []
y_train = []

# Window size
ws = 12 # months

In [ ]:
# Preparing training data
for material_index in range(training_set_scaled.shape[0]):  # Iterate over each row/material
    for i in range(ws, num_columns_for_train - 1):  # Iterating from the beginning of the window to the second-to-last month available in the training set
        X_train.append(training_set_scaled[material_index, i-ws:i]) # Each X_train[material_index] will have a sequence of ws months of data
        y_train.append(training_set_scaled[material_index, i]) # Each y_train[material_index] will be the month following the ws window

xternal Loop (for material_index in range(training_set_scaled.shape[0])): Iterates over each row of the training_set_scaled array, where each row represents a different material.  

Internal Loop (for i in range(ws, num_columns_for_train - 1)): For each material, this line of code is set up to iterate from the index ws to num_columns_for_train - 1. What this means is that you start iterating from the point where you have enough data to form your first data window (ws) and continue until the second-to-last month in the training set, which ensures that you always have future data available to use as y_train. 

X_train.append(...): Captures a window of ws months of data for the current material, starting at i-ws and ending at i (exclusive). 

y_train.append(...): Captures the data for the next immediate month after the ws-month window to use as the target variable for the model.  

This setup is essential to ensure that the model learns to predict the following month based on a window of ws previous months for each individual material.    

In [ ]:
# Convert lists to arrays for training the model
X_train, y_train = np.array(X_train), np.array(y_train)

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train.shape

In the context of a RNN like LSTM in Keras, the input data must be three-dimensional with the dimensions being [number of samples, time steps, features per step].  
This means we have 153,560 training examples, each with 12 time steps. Each time step consists of only one value, the invoiced amount, so X_train.shape = (153560, 12, 1).

# Model Training (Treinamento do Modelo) #

In [ ]:
# Define EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, min_delta=0.0001, mode='min', verbose=1, restore_best_weights=True)

- monitor: Defines which metric the callback should monitor. Validation loss (val_loss) is common, but you can also monitor accuracy (val_accuracy) if relevant.  

- patience: How many epochs to wait after the last improvement before stopping the training.  

- min_delta: The minimum change in the monitored metric (in this case 'val_loss') to be considered an improvement. This basically means "how much better the metric needs to be for it to be considered truly better."  

- mode: Defines whether the training should stop when the monitored metric stops increasing (max) or stops decreasing (min).  

- verbose: Displays log messages.  

- restore_best_weights: If True, the model returns with the weights from the point where it achieved the best performance on the monitored metric. This is useful because the model may have overfitted after the best performance point before actually stopping training.  

Adding Early Stopping not only helps prevent overfitting but also optimizes the use of computational resources by avoiding unnecessarily long training.  

In [ ]:
# LSTM Model
model = Sequential()

model.add(LSTM(units=60, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))

model.add(LSTM(units=60, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=60, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=60))
model.add(Dropout(0.2))

model.add(Dense(units=1))

model.compile(optimizer= 'adam', loss= 'mean_squared_error')
model.fit(X_train, y_train, epochs=100, batch_size= 32, validation_split=0.2, callbacks=[early_stopping])

When the input_shape in the first LSTM layer is configured:     

X_train.shape[1]: This refers to the second dimension of the X_train array, which in this case is the number of time steps per sequence (12). This value tells the model how many time steps each input sequence contains.  
X_train.shape[2]: This refers to the third dimension of the X_train array, which would be the number of features at each time step. Since each time step contains only one value (the sales amount per month for a given material), this value is 1.    

These indices ([1] and [2]) are used because the first index ([0]) represents the total number of sequences or samples, which is not part of the LSTM input_shape. The input_shape specifically needs to know the shape of each individual input sequence, excluding the dimension that counts the total number of sequences. 

In [ ]:
# Plot the errors 
plt.plot(range(len(model.history.history['loss'])), model.history.history['loss'])
plt.xlabel('Epoch Number')
plt.ylabel('Loss')
plt.show()

In [ ]:
# Predictions for all materials
predictions_all_materials = []

# Loop over each material
for material_index in range(training_set_scaled.shape[0]):
    batch_one = training_set_scaled[material_index, -ws:, np.newaxis] # Get the last window of data for the current material
    batch_new = batch_one.reshape((1, ws, 1))
    
    prediction_test = []

    # Predict the next 12 months
    for i in range(12): 
        first_pred = model.predict(batch_new)[0]
        prediction_test.append(first_pred)
        batch_new = np.append(batch_new[:, 1:, :], [[first_pred]], axis=1) # Update the batch to include the new prediction
    
    # Reshape predictions to fit the scaler and inverse transform
    prediction_test = np.array(prediction_test).reshape(1, -1)
    predictions = sc.inverse_transform(prediction_test)
    predictions_all_materials.append(predictions.flatten())

# Convert list to array for easier handling
predictions_all_materials = np.array(predictions_all_materials)

1) for material_index in range(training_set_scaled.shape[0]):  
- training_set_scaled.shape[0] gets the number of rows in the scaled dataset, which corresponds to the number of materials.  
- for material_index in range(...) iterates over each row, where each row represents a different material.   

2) batch_one = training_set_scaled[material_index, -ws:, np.newaxis] & batch_new = batch_one.reshape((1, ws, 1)):  
- training_set_scaled[material_index, -ws:, np.newaxis] selects the last ws (12) months of data for the current material, adding a new dimension to make it compatible with the input expected by the LSTM.  
- .reshape((1, ws, 1)) reshapes the data to have the form (1, ws, 1), where 1 is the number of sequences (batches), ws is the number of time steps per sequence, and the last 1 is the number of features per time step.  

3) for i in range(12):   
    first_pred = model.predict(batch_new)[0]  
    prediction_test.append(first_pred)  
    batch_new = np.append(batch_new[:, 1:, :], [[first_pred]], axis=1):  

- for i in range(12): iterates 12 times to generate predictions for the next 12 months.  
- model.predict(batch_new)[0] makes a prediction using the current batch and takes the first element of the result (since the result is a batch, we need the first element).  
- prediction_test.append(first_pred) adds the prediction to the prediction vector of the current material.  
- np.append(...) updates the batch_new by removing the oldest month and adding the latest prediction at the end, keeping the batch size constant at 12 months.  

4) prediction_test = np.array(prediction_test).reshape(1, -1)  
predictions = sc.inverse_transform(prediction_test)  
predictions_all_materials.append(predictions.flatten()):  
- np.array(prediction_test).reshape(1, -1) converts the list of predictions into a NumPy array and reshapes it for compatibility with the inverse_transform method.  
- sc.inverse_transform(prediction_test) applies the inverse transformation to convert the predictions back to the original data scale.  
- predictions.flatten() flattens the resulting array to remove extra dimensions, and predictions_all_materials.append(...) adds the scaled predictions to the main vector that stores the predictions for all materials.   

5) predictions_all_materials = np.array(predictions_all_materials):  
- Converts the final list that contains the predictions for all materials into a NumPy array to facilitate subsequent handling of the data.  

In [ ]:
# Extracting the data for a single material
actual = test_set[0, :]  # Adjust indices as needed
predicted = predictions_all_materials[0, :]

# Plot Predictions
plt.figure(figsize=(10, 5))
plt.plot(actual, color='red', label='Actual Values')
plt.plot(predicted, color='blue', label='Predicted Values')
plt.title('Sales Forecast for Material X')
plt.xlabel('Month')
plt.ylabel('Invoiced Quantity')
plt.xticks(ticks=np.arange(len(actual)), labels=[f'Month {i+1}' for i in range(len(actual))])
plt.legend()
plt.show()

# Evaluate Model (Avaliação do Modelo) #

In [ ]:
# Create lists to store the results
rmse_list = []
r_square_list = []
material_ids = [f'Material_{i+1}' for i in range(predictions_all_materials.shape[0])]

# Calculate metrics for each material
for i in range(test_set.shape[0]):
    rmse = math.sqrt(mean_squared_error(test_set[i], predictions_all_materials[i]))
    r_square = r2_score(test_set[i], predictions_all_materials[i])
    rmse_list.append(rmse)
    r_square_list.append(r_square)
    print(f'{material_ids[i]} - RMSE: {rmse}, R-Square: {r_square}')

# Create a DataFrame to better visualize the results
df_metrics = pd.DataFrame({
    'Material': material_ids,
    'RMSE': rmse_list,
    'R-Square': r_square_list
})

print(df_metrics)


In [ ]:
def mean_absolute_percetage_error_individual(y_true, y_pred):
    mape_list = []
    for i in range(y_true.shape[0]):  # Iterates over each row/material
        true, pred = np.array(y_true[i]), np.array(y_pred[i])
        mape = np.mean(np.abs((true - pred) / true)) * 100 if np.any(true) else np.nan  # Avoids division by zero
        mape_list.append(mape)
    return mape_list

# Calculates the MAPE for each material
mape_values = mean_absolute_percetage_error_individual(test_set, predictions_all_materials)
material_ids = [f'Material_{i+1}' for i in range(predictions_all_materials.shape[0])]

# Prints the MAPE for each material
for material, mape in zip(material_ids, mape_values):
    print(f'{material} - MAPE: {mape:.2f}%')

# Create a DataFrame to better visualize the results
df_mape = pd.DataFrame({
    'Material': material_ids,
    'MAPE': mape_values
})

print(df_mape)


In [ ]:
# Assuming 'predictions_all_materials' is a list of lists, where each sublist is a series of predictions for a material
predictions_all_materials = np.array(predictions_all_materials)

# Create a DataFrame from the numpy array
# Assume you have a list or array of identifiers for each row (material)
material_ids = [f'Material_{i+1}' for i in range(predictions_all_materials.shape[0])]

# Create a DataFrame with column names representing future monthly periods (adjust as needed)
future_months = [f'Month_{i+1}' for i in range(predictions_all_materials.shape[1])]
df_predictions = pd.DataFrame(predictions_all_materials, index=material_ids, columns=future_months)

# Export the DataFrame to an Excel file
df_predictions.to_excel('predictions_all_materials.xlsx', index=True)

# Display the created DataFrame
df_predictions